In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
# %matplot inline
import numpy as np
from numpy.random import mtrand
import pandas as pd
from sklearn.model_selection import  train_test_split
from scipy.spatial.transform import Rotation
import h5py as h5

## Make Plane

In [6]:
def make_axis_aligned_plane_cluster(n_samples, ax1_center, ax1_range, ax2_center, ax2_range, ax3_value=0.0, noise=None, axes='yz', seed=None, gaussian=True):
    generator = mtrand.RandomState(seed)
    if gaussian:
        ax1 = generator.normal(ax1_center, ax1_range, n_samples)
        ax2 = generator.normal(ax2_center, ax2_range, n_samples)
    else:
        ax1 = generator.uniform(ax1_center - ax1_range, ax1_center + ax1_range, n_samples)
        ax2 = generator.uniform(ax2_center - ax2_range, ax2_center + ax2_range, n_samples)
    X = np.zeros(shape=(n_samples, 3), dtype=np.float64)
    _axes = axes.lower()
    if _axes == 'yz':
        X[:, 0], X[:, 1], X[:, 2] = ax3_value, ax1, ax2
    elif _axes == 'xz':
        X[:, 0], X[:, 1], X[:, 2] = ax1, ax3_value, ax2
    elif _axes == 'xy':
        X[:, 0], X[:, 1], X[:, 2] = ax1, ax2, ax3_value
    else:
        raise ValueError(f'Invalid axes "{_axes}"')
    if noise is not None:
        X += noise * generator.standard_normal(size=(n_samples, 3))
    return X

## Make S-Curve

In [145]:
def make_s_curve_clustered(n_samples, t_center, t_range, y_center, y_range, noise=None, seed=None, gaussian=True):
    generator = mtrand.RandomState(seed)
    if gaussian:
        t = 3 * np.pi * (generator.normal(t_center, t_range, size=(1, n_samples)) - 0.5)
        y = 2.0 * generator.normal(y_center, y_range, size=n_samples)
    else:
        t = 3 * np.pi * (generator.uniform(t_center - t_range, t_center + t_range, size=(1, n_samples)) - 0.5)
        y = 2.0 * generator.uniform(y_center - y_range, y_center + y_range, size=n_samples)
    X = np.empty(shape=(n_samples, 3), dtype=np.float64)
    X[:, 0] = np.sin(t)
    X[:, 1] = y
    X[:, 2] = np.sign(t) * (np.cos(t) - 1)
    X += noise * generator.standard_normal(size=(3, n_samples)).T
    
    return X

## Make Swiss Roll

In [8]:
def make_swiss_roll_clustered(n_samples, t_center, t_range, y_center, y_range, noise=None, seed=None):
    generator = mtrand.RandomState(seed)
    t = 1.5 * np.pi * (1 + 2 * generator.normal(t_center, t_range, size=n_samples))
    y = 10 * generator.normal(y_center, y_range, size=n_samples)
    x = t * np.cos(t)
    z = t * np.sin(t)
    X = np.vstack((x, y, z))
    X += noise * generator.standard_normal(size=(3, n_samples))
    X*=0.2
    X = X.T

    return X

## Make Cylinder

In [57]:
def make_cylinder_clustered(n_samples, t_center, t_range, y_center, y_range, noise=None, seed=None, gaussian=False):
    generator = mtrand.RandomState(seed)
    if gaussian:
        t = 2 * np.pi * (generator.normal(t_center, t_range, size=n_samples) - 0.5)
        y = 10 * generator.normal(y_center, y_range, size=n_samples)
    else:
        t = 2 * np.pi * (generator.uniform(t_center - t_range, t_center + t_range, size=n_samples) - 0.5)
        y = 10 * generator.uniform(y_center - y_range, y_center + y_range, size=n_samples)
    x = np.cos(t)
    z = np.sin(t)
    X = np.vstack((x, y, z))
    X += noise * generator.standard_normal(size=(3, n_samples))
    X*=0.2
    X = X.T

    return X

## save data

In [10]:
def save_prop_data(X, y, name):
    with h5.File(f'datasets/synth/{name}_prop.h5', 'w') as f:
        gE=f.create_group('E')
        gO=f.create_group('O')
        props=[0.9, 0.7, 0.5, 0.3]
        for i,prop in enumerate(props):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=prop, random_state=i, stratify=y)
            gE.create_dataset(f'X{i}', data=X_train)
            gE.create_dataset(f'y{i}', data=y_train)
            gO.create_dataset(f'X{i}', data=X_test)
            gO.create_dataset(f'y{i}', data=y_test)

def save_dist_data(X_train, X_test, y_train, y_test, name):
    with h5.File(f'datasets/synth/{name}_dist.h5', 'w') as f:
        gE=f.create_group('E')
        gO=f.create_group('O')
        gE.create_dataset('X0', data=X_train)
        gE.create_dataset('y0', data=y_train)
        gO.create_dataset('X0', data=X_test)
        gO.create_dataset('y0', data=y_test)


## Plane * 3

In [11]:
# clusters_p = [
#     {'n_samples':200, 'ax1_center':0.0, 'ax1_range':0.15, 'ax2_center':0.0, 'ax2_range':0.15, 'noise':0.01, 'ax3_value':0, 'axes':'yz'},
#     {'n_samples':200, 'ax1_center':0.0, 'ax1_range':0.15, 'ax2_center':1.0, 'ax2_range':0.15, 'noise':0.01, 'ax3_value':0, 'axes':'yz'},
#     {'n_samples':200, 'ax1_center':0.5, 'ax1_range':0.15, 'ax2_center':0.5, 'ax2_range':0.15, 'noise':0.01, 'ax3_value':0, 'axes':'yz'},
#     {'n_samples':200, 'ax1_center':1.0, 'ax1_range':0.15, 'ax2_center':0.0, 'ax2_range':0.15, 'noise':0.01, 'ax3_value':0, 'axes':'yz'},
#     {'n_samples':200, 'ax1_center':1.0, 'ax1_range':0.15, 'ax2_center':1.0, 'ax2_range':0.15, 'noise':0.01, 'ax3_value':0, 'axes':'yz'},
# ]

clusters_p = [
    {'n_samples':350, 'ax1_center':0.0, 'ax1_range':1.2, 'ax2_center':-1.0, 'ax2_range':0.3, 'noise':0.01, 'ax3_value':0.0, 'axes':'xz', 'gaussian': False},
    {'n_samples':350, 'ax1_center':0.0, 'ax1_range':1.2, 'ax2_center': 0.0, 'ax2_range':0.3, 'noise':0.01, 'ax3_value':0.0, 'axes':'xz', 'gaussian': False},
    {'n_samples':350, 'ax1_center':0.0, 'ax1_range':1.2, 'ax2_center': 1.0, 'ax2_range':0.3, 'noise':0.01, 'ax3_value':0.0, 'axes':'xz', 'gaussian': False},
]

n_samples_single = sum([c['n_samples'] for c in clusters_p])
X_p=[]
y_p=[]
clusters_ps = clusters_p + clusters_p + clusters_p
for cluster in clusters_ps:
    Xp = make_axis_aligned_plane_cluster(**cluster)
    X_p.append(Xp)
    y_p.append(np.ones(Xp.shape[0]) * len(X_p))

X = np.vstack(X_p)
y = np.concatenate(y_p)

X1 = X[:n_samples_single]
y1 = y[:n_samples_single]

X2 = X[n_samples_single:n_samples_single * 2]
y2 = y[n_samples_single:n_samples_single * 2]

X3 = X[n_samples_single * 2:]
y3 = y[n_samples_single * 2:]

X1 = Rotation.from_euler('xz', [90,90], degrees=True).apply(X1)
# X1[:, [0, 2]] += np.array([-0.1, -0.5])

X2 = Rotation.from_euler('zx', [90,90], degrees=True).apply(X2)

# X3 = Rotation.from_euler('z', 90, degrees=True).apply(X3)
# X3[:, [0, 2]] += np.array([0.1, 0.5])

X = np.vstack([X1, X2, X3])
y = np.concatenate([y1, y2, y3])
save_prop_data(X, y, 'syn1')

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, train_size=0.7, random_state=100, stratify=y1)
X1_test[:, [0, 2]] += [0.5, 0.1]
X1_test[:, [1, 2]] *= 1.2
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.7, random_state=100, stratify=y2)
X2_test[:, [1, 2]] += [0.5, 0.5]
X2_test[:, [1, 2]] *= 0.8
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, train_size=0.7, random_state=100, stratify=y3)
X3_test[:, [0, 2]] += [-0.5,-0.1]
X3_test[:, [1, 2]] *= 1.2
X_train = np.vstack([X1_train, X2_train, X3_train])
X_test = np.vstack([X1_test, X2_test, X3_test])
y_train = np.concatenate([y1_train, y2_train, y3_train])
y_test = np.concatenate([y1_test, y2_test, y3_test])

save_dist_data(X_train, X_test, y_train, y_test, 'syn1')

fig = plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X_train[:, 0], X_train[:, 1], X_train[:, 2], c='gray', s=1)
plot.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c='red', s=1)
plt.show()

fig=plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X[:,0], X[:,1], X[:,2], c=y, s=2)
plt.show()

## Swiss Roll + Plane

In [12]:
X_s=[]
y_s=[]

clusters_s = [
    {'n_samples':200, 't_center':0.1, 't_range':0.05, 'y_center': 0.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.1, 't_range':0.05, 'y_center': 1.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.3, 't_range':0.05, 'y_center': 0.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.3, 't_range':0.05, 'y_center': 1.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.5, 't_range':0.05, 'y_center': 0.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.5, 't_range':0.05, 'y_center': 1.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.7, 't_range':0.05, 'y_center': 0.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.7, 't_range':0.05, 'y_center': 1.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.9, 't_range':0.05, 'y_center': 0.0, 'y_range':0.3, 'noise':0.01},
    {'n_samples':200, 't_center':0.9, 't_range':0.05, 'y_center': 1.0, 'y_range':0.3, 'noise':0.01},
]
for cluster in clusters_s:
    Xs = make_swiss_roll_clustered(**cluster)
    X_s.append(Xs)
    y_s.append(np.ones(Xs.shape[0]) * len(X_s))
X1=np.vstack(X_s)
y1=np.concatenate(y_s)

clusters_p = [
    {'n_samples':300, 'ax1_center':0.0, 'ax1_range':1.2, 'ax2_center':-1.0, 'ax2_range':0.4, 'noise':0.02, 'ax3_value':0.6, 'axes':'xz', 'gaussian': False},
    {'n_samples':300, 'ax1_center':0.0, 'ax1_range':1.2, 'ax2_center': 0.0, 'ax2_range':0.4, 'noise':0.02, 'ax3_value':0.6, 'axes':'xz', 'gaussian': False},
    {'n_samples':300, 'ax1_center':0.0, 'ax1_range':1.2, 'ax2_center': 1.0, 'ax2_range':0.4, 'noise':0.02, 'ax3_value':0.6, 'axes':'xz', 'gaussian': False},
]
n_samples_single = sum([c['n_samples'] for c in clusters_p])
X_p=[]
y_p=[]
clusters_ps = clusters_p
for cluster in clusters_ps:
    Xp = make_axis_aligned_plane_cluster(**cluster)
    X_p.append(Xp)
    y_p.append(np.ones(Xp.shape[0]) * (len(X_s) + len(X_p)))

Xp=np.vstack(X_p)
yp=np.concatenate(y_p)

X2=Xp[:n_samples_single]
X2[:, [0,2]] *= [2.0]
y2=yp[:n_samples_single]

X = np.vstack([X1, X2])
y = np.concatenate([y1, y2])

save_prop_data(X, y, 'syn2')

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, train_size=0.7, random_state=100, stratify=y1)
X1_test[:, 1] += 2
X1_test[:, [1, 2]] *= 0.9
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.7, random_state=100, stratify=y2)
# X2_test[:, [1, 2]] += [0.5, 0.5]
X2_test[:, [1, 2]] *= 0.8

X_train = np.vstack([X1_train, X2_train])
X_test = np.vstack([X1_test, X2_test])
y_train = np.concatenate([y1_train, y2_train])
y_test = np.concatenate([y1_test, y2_test])

save_dist_data(X_train, X_test, y_train, y_test, 'syn2')

fig = plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X_train[:, 0], X_train[:, 1], X_train[:, 2], c='gray', s=1)
plot.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c='red', s=1)
plt.show()


fig=plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X[:,0], X[:,1], X[:,2], c=y)
plt.show()

## SwissRoll * 2 + Plane

In [14]:

clusters_s = [
    {'n_samples':200, 't_center':0.2 , 't_range':0.07, 'y_center':-0.5, 'y_range':0.3, 'noise':0.02},
    {'n_samples':200, 't_center':0.25, 't_range':0.07, 'y_center': 0.5, 'y_range':0.3, 'noise':0.02},
    {'n_samples':200, 't_center':0.5 , 't_range':0.07, 'y_center':-0.5, 'y_range':0.3, 'noise':0.02},
    {'n_samples':200, 't_center':0.55, 't_range':0.07, 'y_center': 0.5, 'y_range':0.3, 'noise':0.02},
    {'n_samples':200, 't_center':0.8 , 't_range':0.07, 'y_center':-0.5, 'y_range':0.3, 'noise':0.02},
    {'n_samples':200, 't_center':0.75, 't_range':0.07, 'y_center': 0.5, 'y_range':0.3, 'noise':0.02},
]
X_1=[]
y_1=[]
for cluster in clusters_s:
    Xs = make_swiss_roll_clustered(**cluster)
    X_1.append(Xs)
    y_1.append(np.ones(Xs.shape[0]) * len(X_1))

X1=np.vstack(X_1)
# X1=Rotation.from_euler('xy', [180, 90], degrees=True).apply(X1)
X1[:, 0] += -3.0
y1=np.concatenate(y_1)

X_2=[]
y_2=[]
for cluster in clusters_s:
    Xs = make_swiss_roll_clustered(**cluster)
    X_2.append(Xs)
    y_2.append(np.ones(Xs.shape[0]) * (len(X_2) + len(X_1)))

X2 = np.vstack(X_2)
X2=Rotation.from_euler('z', 180, degrees=True).apply(X2)
X2[:, 0] += 3.0
y2 = np.concatenate(y_2)

clusters_s = [
    {'n_samples':300, 'ax1_center':0.0, 'ax1_range':2.5, 'ax2_center':-1.0, 'ax2_range':0.4, 'noise':0.02, 'ax3_value':0.0, 'axes':'xz', 'gaussian': False},
    {'n_samples':300, 'ax1_center':0.0, 'ax1_range':2.5, 'ax2_center': 0.0, 'ax2_range':0.4, 'noise':0.02, 'ax3_value':0.0, 'axes':'xz', 'gaussian': False},
    {'n_samples':300, 'ax1_center':0.0, 'ax1_range':2.5, 'ax2_center': 1.0, 'ax2_range':0.4, 'noise':0.02, 'ax3_value':0.0, 'axes':'xz', 'gaussian': False},
]

X_3=[]
y_3=[]
for cluster in clusters_s:
    Xs = make_axis_aligned_plane_cluster(**cluster)
    
    X_3.append(Xs)
    y_3.append(np.ones(Xs.shape[0]) * (len(X_3) + len(X_2) + len(X_1)))

X3 = np.vstack(X_3)
# X3=Rotation.from_euler('z', 90, degrees=True).apply(X3)
X3[:, [0,2]] *= 2
y3 = np.concatenate(y_3)

X=np.vstack([X1, X2, X3])
y=np.concatenate([y1, y2, y3])

save_prop_data(X, y, 'syn3')

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, train_size=0.7, random_state=100, stratify=y1)
X1_test[:, 1] *= 0.7
X1_test[:, [1, 2]] += [ 2.0, -0.1]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.7, random_state=100, stratify=y2)
X2_test[:, 1] *= 1.3
X2_test[:, [1, 2]] += [-2.0, -0.1]
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, train_size=0.7, random_state=100, stratify=y3)
# X3_test[:, [0, 2]] += [-0.3,-1]
X3_test[:, [1, 2]] *= 0.7
X_train = np.vstack([X1_train, X2_train, X3_train])
X_test = np.vstack([X1_test, X2_test, X3_test])
y_train = np.concatenate([y1_train, y2_train, y3_train])
y_test = np.concatenate([y1_test, y2_test, y3_test])

save_dist_data(X_train, X_test, y_train, y_test, 'syn3')

fig = plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X_train[:, 0], X_train[:, 1], X_train[:, 2], c='gray', s=1)
plot.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c='red', s=1)
plt.show()


fig=plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X1[:,0], X1[:,1], X1[:,2], c='red', s=2)
plot.scatter(X2[:,0], X2[:,1], X2[:,2], c='blue', s=2)
plot.scatter(X3[:,0], X3[:,1], X3[:,2], c='green', s=2)
plt.show()


## S-Curve * 2

In [249]:

cluster_c1 = [
    {'n_samples':200, 't_center':0.25, 't_range':0.05, 'y_center':0.0, 'y_range':0.5, 'noise':0.01, 'gaussian': False},
    {'n_samples':200, 't_center':0.4 , 't_range':0.04, 'y_center':0.0, 'y_range':0.5, 'noise':0.01, 'gaussian': False},
    {'n_samples':200, 't_center':0.5 , 't_range':0.04, 'y_center':0.0, 'y_range':0.5, 'noise':0.01, 'gaussian': False},
    {'n_samples':200, 't_center':0.6 , 't_range':0.04, 'y_center':0.0, 'y_range':0.5, 'noise':0.01, 'gaussian': False},
    {'n_samples':200, 't_center':0.75, 't_range':0.05, 'y_center':0.0, 'y_range':0.5, 'noise':0.01, 'gaussian': False},
]
cluster_c2 = [
    {'n_samples':150, 't_center':0.3, 't_range':0.17, 'y_center':-0.3, 'y_range':0.10, 'noise':0.01, 'gaussian': False},
    {'n_samples':150, 't_center':0.7, 't_range':0.17, 'y_center':-0.3, 'y_range':0.10, 'noise':0.01, 'gaussian': False},
    {'n_samples':150, 't_center':0.3, 't_range':0.17, 'y_center': 0.0, 'y_range':0.10, 'noise':0.01, 'gaussian': False},
    {'n_samples':150, 't_center':0.7, 't_range':0.17, 'y_center': 0.0, 'y_range':0.10, 'noise':0.01, 'gaussian': False},
    {'n_samples':150, 't_center':0.3, 't_range':0.17, 'y_center': 0.3, 'y_range':0.10, 'noise':0.01, 'gaussian': False},
    {'n_samples':150, 't_center':0.7, 't_range':0.17, 'y_center': 0.3, 'y_range':0.10, 'noise':0.01, 'gaussian': False},
]
X=[]
y=[]
cluster_cs = cluster_c1 + cluster_c2
for cluster in cluster_cs:
    x=make_s_curve_clustered(**cluster)
    X.append(x)
    y.append(np.ones(x.shape[0]) * len(X))

X=np.vstack(X)
y=np.concatenate(y)

save_prop_data(X, y, 'syn4')

n_samples_single=sum([c['n_samples'] for c in cluster_c1])

X1=X[:n_samples_single]
y1=y[:n_samples_single]
X2=X[n_samples_single:]
X2 = Rotation.from_euler('z', 180, degrees=True).apply(X2)
y2=y[n_samples_single:]

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, train_size=0.7, random_state=100, stratify=y1)
X1_test[:, 2] *= 0.9
X1_test[:, [1, 2]] += [ 0.2, -0.1]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.7, random_state=100, stratify=y2)
X2_test[:, 2] *= 1.1
X2_test[:, [1, 2]] += [-0.2, -0.1]

X_train = np.vstack([X1_train, X2_train])
X_test = np.vstack([X1_test, X2_test])
y_train = np.concatenate([y1_train, y2_train])
y_test = np.concatenate([y1_test, y2_test])

save_dist_data(X_train, X_test, y_train, y_test, 'syn4')

# fig = plt.figure()
# plot = fig.add_subplot(111, projection='3d')
# plot.scatter(X_train[:, 0], X_train[:, 1], X_train[:, 2], c='gray')
# plot.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c='red')
# plt.show()

fig=plt.figure()
plot = fig.add_subplot(111, projection='3d')
plot.scatter(X1[:,0], X1[:,1], X1[:,2], c=y1)
plot.scatter(X2[:,0], X2[:,1], X2[:,2], c=y2)
plt.show()

## Projection Test

In [2]:
with h5.File(f'datasets/synth/syn4_dist.h5', 'r') as f:
    X_train = f['E']['X0'][:]
    y_train = f['E']['y0'][:]
    X_test  = f['O']['X0'][:]
    y_test  = f['O']['y0'][:]

    print("X_train shape: ", X_train.shape)
    print("X_test shape: ", X_test.shape)

X_train shape:  (1330, 3)
X_test shape:  (570, 3)


In [6]:
from methods.methods import Methods

methods = Methods(verbose=True)

projectors = [
    methods.get('pca'),
    methods.get('kisomap'),
    methods.get('ptsne22'),
    methods.get('ktsne'),
    methods.get('cdr'),
    methods.get('pumap'),
]

projs = []

for p in projectors:
    p.fit(X_train)
    proj=p.transform(X_train)
    proj_test=p.transform_oos(X_test)
    projs.append((proj, proj_test))


perplexity: 30
calc P


2024-10-10 00:23:46,252 - INFO - Precomputed initialization provided. Ignoring initalization-related parameters.


Trying to put X into GPU
optimizing...


Processing epoch 1000/1000 loss : -0.03602 time : 0.00391s: 100%|██████████| 1000/1000 [00:43<00:00, 22.78it/s]
2024-10-10 00:24:35,464 - INFO - Start train for Visualize
2024-10-10 00:24:49,705 - INFO - successfully compute approximate neighbor_graph
2024-10-10 00:24:51,378 - INFO - Start Training for 1000 Epochs
2024-10-10 00:24:51,379 - INFO - Experiment Configurations: 
Epochs: 1000 Batch Size: 133 
Learning rate: 0.001000 Optimizer: adam

2024-10-10 00:24:51,380 - INFO - Start Training for 1000 Epochs
2024-10-10 00:25:09,612 - INFO - Epoch 100/1000, Train Loss: 1.85744, 
2024-10-10 00:25:27,901 - INFO - Epoch 200/1000, Train Loss: 1.86767, 
2024-10-10 00:25:47,614 - INFO - Epoch 300/1000, Train Loss: 1.89923, 
2024-10-10 00:26:08,428 - INFO - Epoch 400/1000, Train Loss: 1.92492, 
2024-10-10 00:26:29,284 - INFO - Epoch 500/1000, Train Loss: 1.91765, 
2024-10-10 00:26:50,095 - INFO - Epoch 600/1000, Train Loss: 1.82873, 
2024-10-10 00:27:10,870 - INFO - Epoch 700/1000, Train Loss: 1

ParametricUMAP(batch_size=133, loss_report_frequency=1, optimizer=<keras.optimizers.optimizer_v2.adam.Adam object at 0x000002CF12158C10>)
Thu Oct 10 00:28:09 2024 Construct fuzzy simplicial set
Thu Oct 10 00:28:10 2024 Finding Nearest Neighbors
Thu Oct 10 00:28:13 2024 Finished Nearest Neighbor Search
Thu Oct 10 00:28:15 2024 Construct embedding
42/42 [==============================] - 0s 965us/step
Thu Oct 10 00:30:15 2024 Finished embedding
5/5 [==============================] - 0s 1ms/step


In [4]:
fig = plt.figure(figsize=(len(projs) * 3, 3))
for i,(proj, proj_test) in enumerate(projs):
    plot = fig.add_subplot(1, len(projs), i + 1)
    plot.scatter(proj[:,0], proj[:,1], c='gray', alpha=0.2, marker='o', s=3)
    plot.scatter(proj_test[:,0], proj_test[:,1], c=y_test, alpha=0.7, marker='o', s=3)
    plot.set_title(projectors[i].__class__.__name__)
    plot.set_xticks([])
    plot.set_yticks([])
    plot.set_box_aspect(1.0)

plt.show()
# fig.show()
# %matplotlib inline
# %qtconsole

In [5]:
from metrics import Metrics
metrics = Metrics()
df=pd.DataFrame(index=metrics.available())
for i,(proj, proj_test) in enumerate(projs):
    metrics.update_metrics(X_train=X_train, X_train_Embedded=proj, X_test=X_test, X_test_Embedded=proj_test, y_train=y_train, y_test=y_test)
    results=[]
    for metric_name in metrics.available():
        res,_ = metrics.run_single(metric_name)
        results.append(res)
    df[projectors[i].__class__.__name__] = results
        
print(df)

             _pca
t        0.934663
c        0.985649
nh       0.731830
lc       0.386216
sd       0.950563
tp      -0.023261
sc       0.274576
dsc      0.647368
acc_oos  0.684211
acc_e    0.681203
